In [1]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# import ipywidgets as widgets
# from ipywidgets import interact, interact_manual

import traceback
import numpy as np
# import scipy.stats as ss
import itertools
from os import path
import pickle
# from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.preprocessing import PolynomialFeatures, StandardScaler
# from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [30, 13]

import pandas as pd
pd.options.display.width = 500

slam_node_colors = {'gmapping': 'orange', 'slam_toolbox': 'blue', 'hector_slam': 'cyan'}


In [2]:
def load_cache(file_path):
    with open(path.expanduser(file_path), 'rb') as f:
        cache_dict = pickle.load(f)
        return cache_dict['df']

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [23]:
# load data from cache and rename columns
df = load_cache("/mnt/HHD_ext4/ds/performance_modelling/output/global_planning/run_data_per_waypoint_cache.pkl")
df.rename(inplace=True, columns={
    'feasibility_rate_feasibility_rate_of_path': 'fr',
    'planning_time_over_voronoi_distance_normalized_planning_time_for_each_path': 'npt',
    'euclidean_length_over_voronoi_distance_normalised_plan_length_for_each_path': 'npl',
})

# refactor and extract additional parameters and metrics
# df['linear_update'] = df['linear_angular_update'].apply(lambda x: x[0])
# df['beta'] = df['beta'].apply(lambda x: max(x))
# df['norm_translation_error'] = df.translation_error / df.trajectory_length
df['run_id'] = df['run_folder'].apply(lambda x: path.basename(x))
# df['waypoint_index'] = df.sort_values('start_time_cont').groupby(df.run_id).cumcount()
# df['accumulated_trajectory_length_cont'] = df.sort_values('start_time_cont').groupby(df.run_id).trajectory_length.cumsum() - df.trajectory_length
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))

# only keep necessary columns
robot_parameters = [
    'robot_kinematic',
]
component_parameters = [
    'global_planner_name',
    'max_steering_angle',
    'primitives_per_angle',
    'time_out',
]
other_parameters = [
#     'session_id',
#     'run_number',
#     'run_id',
    'environment_name',
]
output_metrics = [
    'fr',
    'npl',
    'npt',
]
df = df[
    robot_parameters +
    component_parameters +
    other_parameters +
    output_metrics
]

print('\n'.join(list(df.columns)))
print(df[component_parameters])

printmd("## Parameters and Their Values")
for c in robot_parameters+component_parameters:
    printmd("{:>35}".format(c), sorted(list(df[c].unique())))

robot_kinematic
global_planner_name
max_steering_angle
primitives_per_angle
time_out
environment_name
fr
npl
npt
      global_planner_name  max_steering_angle  primitives_per_angle  time_out
0       OmplGlobalPlanner                 NaN                   NaN       NaN
1       OmplGlobalPlanner                 NaN                   NaN       NaN
2       OmplGlobalPlanner                 NaN                   NaN       NaN
3       OmplGlobalPlanner                 NaN                   NaN       NaN
4       OmplGlobalPlanner                 NaN                   NaN       NaN
...                   ...                 ...                   ...       ...
64970   OmplGlobalPlanner                 NaN                   NaN       2.0
64971   OmplGlobalPlanner                 NaN                   NaN       2.0
64972   OmplGlobalPlanner                 NaN                   NaN       2.0
64973   OmplGlobalPlanner                 NaN                   NaN       2.0
64974   OmplGlobalPlanner    

## Parameters and Their Values

                    robot_kinematic ['bicycle', 'unicycle']

                global_planner_name ['GlobalPlanner', 'OmplGlobalPlanner', 'SBPLLatticePlanner']

                 max_steering_angle [nan, 20.0, 25.0, 30.0, 50.0]

               primitives_per_angle [nan, 7.0, 9.0, 11.0]

                           time_out [nan, 2.0, 10.0]

In [ ]:
plt.